In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

from keras.datasets import mnist
import tensorflow.keras as kb
from tensorflow.keras import backend
import tensorflow as tf
from sklearn.preprocessing import LabelBinarizer


from plotnine import *

from sklearn.metrics import mean_squared_error, mean_absolute_error, roc_auc_score

from sklearn.linear_model import LinearRegression # Linear Regression Model
from sklearn.preprocessing import StandardScaler #Z-score variables

from sklearn.model_selection import train_test_split # simple TT split cv
from sklearn.model_selection import KFold # k-fold cv


# Regularization in NNs


In CPSC 392, we talked about Regularization through things like LASSO/Ridge penalties, Bagging in Random Forests, and Limiting `max_depth` or `min_samples_leaf` in Decision Trees. All of these methods aim to make our models *less complex* which can move us from an area of high variance/overfitting, to an area where we are NOT overfitting nor underfitting (our main goal.)

<center><img src="https://drive.google.com/uc?export=view&id=1akEMD7_STAkqJcZUI5vc3CW5aU5AeTWR"></center>

However, the models we learned in 392 tended to be pretty simple. Deep Neural Networks are typically *NOT* simple. This complexity allows them to learn complicated relationships between our inputs and our outputs. For example, with linear regression, you're limited to linear relationships between inputs and outputs, neural networks allow you to have non-linear relationships, and the deeper the network the more complicated those relationships can be (remember we talked about Universal Function Approximation?)

Remember the whole *goal* of Machine Learning is to approximate the function $y = f(x)$ that tells you how to take our inputs $x$ and turn them into an accurate guess at our output(s) $y$. That function is $f(x)$. The more complicated $f(x)$ is, the more flexible/complex our model needs to be to approximate it. Models like regression, decision trees...etc. often aren't flexible enough to solve complicated problems like the one's we'll encounter later in the class. BUT Neural Networks can be!

However, we know that when a model is TOO complex/flexible, it can overfit...

<center><img src="https://jashrathod.github.io/assets/img/optimal-fit.png" alt = "from:https://jashrathod.github.io/2021-09-30-underfitting-overfitting-and-regularization/" width = "400px" ></center>


So we often need regularization to make sure that our models are not overfitting. Remember that **overfitting is when the model is so complex that it can pick up on noise/quirks in the training data that aren't there in other samples of data** so our goal when regularizing is to prevent this from happening. 

We talked about *multiple* methods of Regularization for NNs:

- Penalization
- Bagging or Model Averaging
- Dropout
- Early Stopping
- Batch Normalization



# Regularization in keras

## Penalization

We can add penalization to any/all our layers using the `kernel_regularizer` argument. While you can create your own custom regularizers, we'll focus on `L1`, `L2` and ElasticNet aka `L1L2`. Let's take the MNIST model we built in Classwork 5 and add penalties to the layers. 

In [2]:
# load and rescale the data so that the pixel values are between 0-1

# load data
((trainX, trainY), (testX, testY)) = mnist.load_data()

# reshape datta to be individual columns instead of image matrix
trainX = trainX.reshape((trainX.shape[0], 28 * 28 * 1))
testX = testX.reshape((testX.shape[0], 28 * 28 * 1))

# rescale data to be 0-1 instead of 0-255
trainX = trainX.astype("float32") / 255.0
testX = testX.astype("float32") / 255.0

# change the labels to be in the correct format
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

In [3]:
from tensorflow.keras import regularizers

# build structure of the model
model = kb.Sequential([
    kb.layers.Dense(250, input_shape =[784]), #input
    kb.layers.Dense(100),
    kb.layers.Dense(50),
    kb.layers.Dense(30),
    kb.layers.Dense(20),
    kb.layers.Dense(10),
    kb.layers.Dense(10, activation = "softmax") #output
])

# compile model
model.compile(loss="categorical_crossentropy", optimizer=kb.optimizers.SGD(0.01),
	metrics=["accuracy"])

#fit the model (same as SKlearn)
model.fit(trainX,trainY, epochs = 100, validation_data=(testX, testY))

Epoch 1/100


2023-03-02 11:53:29.764553: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


1875/1875 [==============================] - 5s 2ms/step - loss: 0.5020 - accuracy: 0.8526 - val_loss: 0.3431 - val_accuracy: 0.9017
Epoch 2/100
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3270 - accuracy: 0.9074 - val_loss: 0.3068 - val_accuracy: 0.9143
Epoch 3/100
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3060 - accuracy: 0.9132 - val_loss: 0.2936 - val_accuracy: 0.9188
Epoch 4/100
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2946 - accuracy: 0.9170 - val_loss: 0.3042 - val_accuracy: 0.9101
Epoch 5/100
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2892 - accuracy: 0.9189 - val_loss: 0.3153 - val_accuracy: 0.9083
Epoch 6/100
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2842 - accuracy: 0.9214 - val_loss: 0.2909 - val_accuracy: 0.9172
Epoch 7/100
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2817 - accuracy: 0.9203 - val_loss: 0.2918 - val_accuracy: 0.91

KeyboardInterrupt: 

Using this method of adding a penalty on the weights will use the default **regularization strength**. Remember that regularization strength tells the model how MUCH to penalize large weights. If regularization strength is 0, then there's no penalty. If it's greater than 0, there will be more and more penalization as regularization strength gets bigger. 

Typically these defaults are pretty good (check them out [here](https://keras.io/api/layers/regularizers/)). BUT if you'd like more control over the regularization strength, you can also pass a `regulizers` object to the `kernel_regularizer` argument. 

In [4]:
from tensorflow.keras import regularizers

# build structure of the model
model = kb.Sequential([
    kb.layers.Dense(250, input_shape =[784]), #input
    kb.layers.Dense(100, kernel_regularizer = "l1"),
    kb.layers.Dense(50, kernel_regularizer = "l2"),
    kb.layers.Dense(30, kernel_regularizer = regularizers.L1L2(l1 = 0.0000001, l2 = 0.0001)),
    kb.layers.Dense(20, kernel_regularizer = regularizers.L1(0.01)),
    kb.layers.Dense(10, kernel_regularizer = regularizers.L2(0.002)),
    kb.layers.Dense(10, activation = "softmax") #output
])

# compile model
model.compile(loss="categorical_crossentropy", optimizer=kb.optimizers.SGD(0.01),
	metrics=["accuracy"])

#fit the model (same as SKlearn)
model.fit(trainX,trainY, epochs = 1, validation_data=(testX, testY))

1875/1875 [==============================] - 5s 2ms/step - loss: 5.9507 - accuracy: 0.8363 - val_loss: 1.6470 - val_accuracy: 0.8621


## Bagging 

When we use Bagging, our goal is to have many *slightly different* models that we then average together to make a prediction. Traditionaly (like in Random Forests) we implement this by **bootstrapping** samples of data to train on. Bootstrapping is sampling *with replacement* so that each bootstrapped sample may have some data poitns that occur multiple times, and others that do not occur at all. Thus each model is trained on *slightly* different data.

This helps prevent overfitting because while each model might overfit to the noise/quirks in the sample it sees, no two models will overfit to the SAME noise/quirks because they're seeing different data. Thus, what the models have in common are the broad/general patterns that apply to *any* sample of data. 

It turns out that NNs don't *necessarily* need bagging in order to get several *slightly different* networks. NNs are sensitive to things like:

- the initial random weights selected before we start training
- which points end up in which mini-batch when training
- etc...

And thus it may be possible to train several networks without bagging and still end up with an ensemble of slightly different networks (you can still use bagging though, but its not easy with Keras).


We won't build these too often, so I won't spend time going over it here, but [here's](https://www.adriangb.com/scikeras/stable/notebooks/Meta_Estimators.html#4.-Bagging-ensemble) an example.

## Dropout

Dropout is one of the most POPULAR methods of regularization in NNs because it is easy to implement, works with pretty much any NN structure, and is very effective at reducing overfitting. Dropout works on the same principles as *Bagging* but is more efficiently implemented.

In Dropout we mask--aka set to 0--some of the outputs of a layer before feeding them into the next layer. We do this randomly and it changes each time we take a pass through the network, so as the model is training, it basically never knows which inputs will not be there...so it can't "put all its eggs in one basket" so to speak. In other words, the model cannot over-rely on any connections, because sometimes those nodes will just be gone. 

In keras, [Dropout](https://keras.io/api/layers/regularization_layers/dropout/) is implemented through a `Dropout()` layer, which you can stick between any two layers in your network. When data passes through the Dropout layer, it will randomly set some of the inputs into the `Dropout()` layer to 0 based on the `rate` argument which is a value between 0 and 1 indicating the fraction of nodes in that layer to set to 0/drop. 0 would mean we do not drop any values. 1 means we drop all values. Typically we'll set this value to something like 0.2, 0.3, or 0.5. 

**Note**: Dropout is only applied during *training*. When making predictions with the model, we do not dropout any nodes. 

 Let's take the MNIST model we built in Classwork 5 and add Dropout to the layers. 



In [10]:

# build structure of the model
model = kb.Sequential([
    kb.layers.Dense(250, input_shape =[784]),
    kb.layers.Dropout(0.5),
    kb.layers.Dense(100),
    kb.layers.Dropout(0.3),
    kb.layers.Dense(50),
    kb.layers.Dropout(0.2),
    kb.layers.Dense(30),
    kb.layers.Dropout(0.3),
    kb.layers.Dense(20),
    kb.layers.Dropout(0.2),
    kb.layers.Dense(10),
    kb.layers.Dense(10, activation = "softmax") #output
])

# compile model
model.compile(loss="categorical_crossentropy", optimizer=kb.optimizers.SGD(0.01),
	metrics=["accuracy"])

#fit the model (same as SKlearn)
model.fit(trainX,trainY, epochs = 3, validation_data=(testX, testY))

Epoch 1/3
1875/1875 [==============================] - 5s 2ms/step - loss: 1.0417 - accuracy: 0.6508 - val_loss: 0.4269 - val_accuracy: 0.8836
Epoch 2/3
1875/1875 [==============================] - 4s 2ms/step - loss: 0.7054 - accuracy: 0.7792 - val_loss: 0.3854 - val_accuracy: 0.8929
Epoch 3/3
1875/1875 [==============================] - 4s 2ms/step - loss: 0.6514 - accuracy: 0.8002 - val_loss: 0.3626 - val_accuracy: 0.8967


## Early Stopping 

Early Stopping regularizes our model by stopping training once the validation loss no longer improves. Remember that the pattern of overfitting shows that overfitting happens when the training loss continues to improve while the validation loss stops improving or gets worse. 

In Keras, [Early Stopping](https://keras.io/api/callbacks/early_stopping/) is implemented through a `callback` which is an object that has access to your model information during training and can make changes to the training process during training. 

 Let's take the MNIST model we built in Classwork 5 and add Early Stopping. 

In [9]:
# tells keras to stop training when the validation loss no longer improves for 3 epochs in a row

from tensorflow.keras import regularizers

callbacks = [tf.keras.callbacks.EarlyStopping(monitor = 'loss', patience = 3)]

# build structure of the model
model = kb.Sequential([
    kb.layers.Dense(250, input_shape =[784]), #input
    kb.layers.Dense(100),
    kb.layers.Dense(50),
    kb.layers.Dense(30),
    kb.layers.Dense(20),
    kb.layers.Dense(10),
    kb.layers.Dense(10, activation = "softmax") #output
])

# compile model
model.compile(loss="categorical_crossentropy", optimizer=kb.optimizers.SGD(0.01),
	metrics=["accuracy"])

#fit the model (same as SKlearn)
model.fit(trainX,trainY, epochs = 3, validation_data=(testX, testY), callbacks = callbacks)

Epoch 1/3
1875/1875 [==============================] - 5s 2ms/step - loss: 0.4988 - accuracy: 0.8511 - val_loss: 0.3266 - val_accuracy: 0.9086
Epoch 2/3
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3277 - accuracy: 0.9073 - val_loss: 0.2936 - val_accuracy: 0.9173
Epoch 3/3
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3083 - accuracy: 0.9119 - val_loss: 0.2913 - val_accuracy: 0.9167


## Batch Norm

Batch Normalization was actually proposed as a way to handle something called "covariate shift" but much like many things in Deep Learning, we discovered by accident that it had a regularizing effect on models. We don't really know why it works, but hey if it works, it works. Some propose that the "noise" inserted into our model by using the *estimate* of the mean and variance for each batch is what causes this regularization. It also can speed up training and allow us to use larger learning rates because it smooths out the loss surface we're trying to descend. 

Batch Norm is similar to z-scoring, but we do it *between* layers. This ensures that the input to each layer in our network is stable. 

Batch Norm first takes the output and centers it based on the estimated mean of the outputs of a layer, scales it based on the estimated variance/standard deviation of the outputs of a layer, and then scaling and shifting it based on learned parameters $\gamma$ and $\beta$ which allow the output to have a different mean ($\beta$) and standard deviation ($\gamma$) than 0 and 1.

Thus we take the outputs $\mathbf{x}$ and turn them into

$$\beta + \gamma * \frac{\mathbf{x} - \mu_B}{\sqrt{(\sigma_B + \epsilon)}}$$

Where $\beta$ is the new mean of the outputs and $\gamma$ is the new standard deviation of the outputs. $\epsilon$ is just a tiny value that helps prevent us from dividing by 0. 

$\gamma$ and $\beta$ are learned during training through backpropagation/gradient descent just like the weights and biases in our model. 

In Keras, we can use a [`BatchNormalization()`](https://keras.io/api/layers/normalization_layers/batch_normalization/) layer in order to implement Batch Normalization.  Let's take the MNIST model we built in Classwork 5 and add Batch Normalization to the layers. 


In [8]:

# build structure of the model
model = kb.Sequential([
    kb.layers.Dense(250, input_shape =[784]), #input
    kb.layers.BatchNormalization(),
    kb.layers.Dense(100),
    kb.layers.BatchNormalization(),
    kb.layers.Dense(50),
    kb.layers.BatchNormalization(),
    kb.layers.Dense(30),
    kb.layers.BatchNormalization(),
    kb.layers.Dense(20),
    kb.layers.BatchNormalization(),
    kb.layers.Dense(10),
    kb.layers.Dense(10, activation = "softmax") #output
])

# compile model
model.compile(loss="categorical_crossentropy", optimizer=kb.optimizers.SGD(0.01),
	metrics=["accuracy"])

#fit the model (same as SKlearn)
model.fit(trainX,trainY, epochs = 3, validation_data=(testX, testY))

Epoch 1/3
1875/1875 [==============================] - 7s 3ms/step - loss: 0.5909 - accuracy: 0.8307 - val_loss: 0.3391 - val_accuracy: 0.9039
Epoch 2/3
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3866 - accuracy: 0.8877 - val_loss: 0.3040 - val_accuracy: 0.9129
Epoch 3/3
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3544 - accuracy: 0.8966 - val_loss: 0.2960 - val_accuracy: 0.9128


# On Your Own

We can use multiple methods of regularization at once (although there are some rules of thumb, like use penalties when your network is small, and Dropout when it is large...). 

Take the MNIST NN and add multiple methods of regularization as you see fit. 

In [16]:
# build structure of the model
model = kb.Sequential([
    kb.layers.Dense(1000, input_shape =[784]),
    kb.layers.BatchNormalization(),
    kb.layers.Dense(950),
    kb.layers.Dense(800),
    kb.layers.BatchNormalization(),
    kb.layers.Dense(550),
    kb.layers.Dense(300),
    kb.layers.BatchNormalization(),
    kb.layers.Dense(100),
    kb.layers.Dense(75),
    kb.layers.BatchNormalization(),
    kb.layers.Dense(50),
    kb.layers.Dense(30),
    kb.layers.BatchNormalization(),
    kb.layers.Dense(20),
    kb.layers.Dense(10),
    kb.layers.Dense(10, activation = "softmax") #output
])

# compile model
model.compile(loss="categorical_crossentropy", optimizer=kb.optimizers.SGD(0.01),
	metrics=["accuracy"])

#fit the model (same as SKlearn)
model.fit(trainX,trainY, epochs = 3, validation_data=(testX, testY), callbacks = callbacks)

Epoch 1/3
1875/1875 [==============================] - 41s 21ms/step - loss: 0.4998 - accuracy: 0.8543 - val_loss: 0.3180 - val_accuracy: 0.9101
Epoch 2/3
 583/1875 [========>.....................] - ETA: 26s - loss: 0.3665 - accuracy: 0.8947